# This notebook can be used to run and test the sciml services
It load the configuration files and all the output of the main topic classifications.
Once you have started the services you can send a message to configure one of the classifiers to wait on a topic
queue and then send it some items to classify. 

In [1]:
import pika
import time
import azure
#from azure.storage.table import TableService
#from azure.storage.blob import BlobService
import pickle
import random
import json


In [2]:
import urllib
def read_config_from_azure_blob(subj):
    base = "http://esciencegroup.blob.core.windows.net/scimlpublic"
    docpath =base+ "/config_"+subj+".json"
    f = urllib.urlopen(docpath)
    doc = f.read()
    z =json.loads(doc)
    subject = z['subject']
    loadset = z['loadset']
    subtopics = []
    for w in z['supertopics']:
        subtopics.extend([(w[0], w[1])])
    return subject, loadset, subtopics

We first load all the output of the main topic classifiers.
these have been stored in files named "dump_subject_subtopic_name.p"

In [3]:

subject, loadset,supertopics =read_config_from_azure_blob("all4")
basepath = "http://esciencegroup.blob.core.windows.net/scimlpublic"
namspath = basepath+"/supertopic_names-"+subject+".p"
groupb = urllib.urlopen(namspath).read()   
groupnames = pickle.loads(str(groupb))
print groupnames
listTable = {gname:[] for gname in groupnames}
for nam in groupnames:      
    path = basepath+"/dump_"+subject+"_subtopic_"+nam+".p"
    bys = urllib.urlopen(path).read()
    listTable[nam] = pickle.loads(str(bys))


[u'Physics', u'math', u'bio', u'compsci', u'finance']


In [4]:
for nam in groupnames:
    print nam + " "+ str(len(listTable[nam]))

Physics 2988
math 1943
bio 817
compsci 1626
finance 453


In [5]:
print listTable["math"][5]

(u'math', u'compsci', '??', 'A double nanohole in a metal film was used to trap nanoparticles (20 nm diameter) and simultaneously record their Raman spectrum using the trapping laser as the excitation source. This allowed for the identification of characteristic Stokes lines for titania and polystyrene nanoparticles, showing the capability for material identification of nanoparticles once trapped. Increased Raman signal is observed for the trapping of multiple nanoparticles. This system combines the benefits of nanoparticle isolation and manipulation with unique identification.', u'Physics', u'Raman Spectroscopy of Single Nanoparticles in a Double-Nanohole Optical   Tweezer System [physics.optics]')


Now for each main topic we create a dictionary indexed by the group name.  the entry is a list of the json style object that the classifier services will expect to see.

In [6]:
jsonitems = {"Physics":[], "bio":[], "math":[], "compsci":[], "finance":[]}
for item in listTable["Physics"]:
    jsonitems["Physics"].extend([("phy", {"rf": item[0], "best": item[1], "cent": item[2], "doc": item[3], 
                      "truetop": item[4], "title": item[5]})])
for item in listTable["bio"]:
    jsonitems["bio"].extend([("bio", {"rf": item[0], "best": item[1], "cent": item[2], "doc": item[3], 
                     "truetop": item[4], "title": item[5]})])
for item in listTable["math"]:
    jsonitems["math"].extend([("math",{"rf": item[0], "best": item[1], "cent": item[2], "doc": item[3], 
                      "truetop": item[4], "title": item[5]})])
for item in listTable["compsci"]:
    jsonitems["compsci"].extend([("compsci",{"rf": item[0], "best": item[1], "cent": item[2], "doc": item[3], 
                      "truetop": item[4], "title": item[5]})])
for item in listTable["finance"]:
    jsonitems["finance"].extend([("finance",{"rf": item[0], "best": item[1], "cent": item[2], "doc": item[3], 
                      "truetop": item[4], "title": item[5]})])


the names of the queues we need to have in the message broker are as follows.
You will notice we have a disconnect with the name for Physics.   it's queue is phy.  sorry
for this inconsistancy.   

In [7]:
queueset = set(["math", "bio", "phy", "finance", "compsci"] )

In [8]:
creds = pika.PlainCredentials('dbg', 'dbgrabbit')
rabbitmq_location = "dbgswarm1.cloudapp.net"

In [9]:
def sendtopic(topic):
    connec = pika.BlockingConnection(pika.ConnectionParameters(rabbitmq_location, 5672,'/',creds))
    chan = connec.channel()
    chan.queue_declare(queue="roles")
    chan.basic_publish(exchange='',
        routing_key="roles",
        body=topic)  
    connec.close()

In [10]:
def sendreset(topic):
    connec = pika.BlockingConnection(pika.ConnectionParameters(rabbitmq_location, 5672,'/',creds))
    chan = connec.channel()
    chan.queue_declare(queue=topic)
    chan.basic_publish(exchange='',
        routing_key=topic,
        body="reset")
    connec.close()

to tell a classifier to start listening on a topic do the following

In [14]:
sendtopic("bio")

if a classifier is already listening on a topic and you want it to stop do the following

In [12]:
sendreset("bio")

this can be used to send a set of items from the main classification into the various queues for processing

In [13]:
connec = pika.BlockingConnection(pika.ConnectionParameters(rabbitmq_location, 5672,'/',creds))
chan = connec.channel()
#for nam in groupnames:
for nam in ["bio"]: #,"math", "compsci","finance", "Physics"]: 
    for x in jsonitems[nam][201:204]:
        print "sending json to pipe ="+x[0]
        z = x[1]
        doc = z["doc"]
        doc = doc.replace("\\", "")
        doc = doc.replace("'","")
        doc = doc.replace("\'","")
        doc = doc.replace('"',"")
        x[1]["doc"]= doc
        y = str(x[1])
        y=y.replace("u'","'")
        y=y.replace("'",'"')
        chan.basic_publish(exchange='',
            routing_key=x[0],
            body=str(y))
            #print " sent startup: "+str(y)
        time.sleep(0.1)
connec.close()

sending json to pipe =bio
sending json to pipe =bio
sending json to pipe =bio


In [16]:
sendreset("bio")